In [1]:
%pip install -r 002-requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
!docker-compose -f docker-images/docker-compose-milvus.yml up -d
!docker run --rm --name postgres0 -d  -p 5438:5432 -e POSTGRES_HOST_AUTH_METHOD=trust postgres

[+] Running 1/0
 ✔ Network milvus          Created                                         0.0s 
 ⠋ Container milvus-minio  Creating                                        0.0s 
 ⠋ Container milvus-etcd   Creating                                        0.0s 
[+] Running 3/3
 ✔ Network milvus                                                                                                                                            Created0.0s 
 ⠿ Container milvus-minio                                                                                                                                    Starting0.1s 
 ⠿ Container milvus-etcd                                                                                                                                     Starting0.1s 
 ✔ Container milvus-standalone                                                                                                                               Created0.0s 
 ! standalone The requested image's platfor

In [3]:
!docker logs postgres0 --tail 6

selecting dynamic shared memory implementation ... posix
selecting default max_connections ... 100
selecting default shared_buffers ... 128MB
selecting default time zone ... Etc/UTC
creating configuration files ... ok
running bootstrap script ... ok


In [4]:
#Connectings to Milvus, BERT and Postgresql
from pymilvus import connections
import psycopg2
connections.connect(host='localhost', port='19530')
conn = psycopg2.connect(host='localhost', port='5438', user='postgres', password='postgres')
cursor = conn.cursor()

In [5]:
TABLE_NAME = "text_collection"
FIELD_NAME = "example_field"

from pymilvus import Collection, CollectionSchema, FieldSchema, DataType

pk = FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True)

field = FieldSchema(name=FIELD_NAME, dtype=DataType.FLOAT_VECTOR, dim=768)
schema = CollectionSchema(fields=[pk,field], description="example collection")

collection = Collection(name=TABLE_NAME, schema=schema)

In [8]:
index_param = {
        "metric_type":"L2",
        "index_type":"IVF_SQ8",
        "params":{"nlist":1024}
    }

collection.create_index(field_name=FIELD_NAME, index_params=index_param)

Status(code=0, message=)

In [9]:
#Deleting previouslny stored table for clean run
drop_table = "DROP TABLE IF EXISTS " + TABLE_NAME
cursor.execute(drop_table)
conn.commit()

try:
    sql = "CREATE TABLE if not exists " + TABLE_NAME + " (ids bigint, title text, text text);"
    cursor.execute(sql)
    conn.commit()
    print("create postgres table successfully!")
except Exception as e:
    print("can't create a postgres table: ", e)

create postgres table successfully!


In [19]:
from sentence_transformers import SentenceTransformer
import pandas as pd
from sklearn.preprocessing import normalize

model = SentenceTransformer('paraphrase-mpnet-base-v2')

# Get article data
data = pd.read_json('news-data/News_Category_Dataset_v3_Small.json', lines=True)

title_data = data['headline'].tolist()
text_data = data['short_description'].tolist()

sentence_embeddings = model.encode(title_data)
sentence_embeddings = normalize(sentence_embeddings)
print(type(sentence_embeddings))

Batches: 100%|██████████| 32/32 [00:09<00:00,  3.53it/s]

<class 'numpy.ndarray'>


In [20]:
em =list(sentence_embeddings)
mr = collection.insert([em])
ids = mr.primary_keys
dicts ={}

In [23]:
import os



def record_temp_csv(fname, ids, title, text):
    with open(fname,'w') as f:
        for i in range(len(ids)):
            line = str(ids[i]) + "|" + title[i] + "|" + text[i] + "\n"
            f.write(line)

def copy_data_to_pg(table_name, fname, conn, cur):
    fname = os.path.join(os.getcwd(),fname)
    try:
        sql = f"COPY {table_name} FROM STDIN (QUOTE E'\u0007', FORMAT 'csv', DELIMITER '|')"
        cursor.copy_expert(sql, open(fname, "r"))
        conn.commit()
        print("Inserted into Postgress Sucessfully!")
    except Exception as e:
        print("Copy Data into Postgress failed: ", e)
        
DATA_WITH_IDS = 'news-data/temp-to-load.csv'   

record_temp_csv(DATA_WITH_IDS, ids, title_data, text_data)
copy_data_to_pg(TABLE_NAME, DATA_WITH_IDS, conn, cursor)

Copy Data into Postgress failed:  current transaction is aborted, commands ignored until end of transaction block



### Cleanup of docker images

In [6]:
%%bash
L_END_DOCKER=0

if [[ ${L_END_DOCKER} -eq 1 ]]; then
    docker-compose -f docker-images/docker-compose-milvus.yml down --remove-orphans
    docker stop postgres0
fi

### Articles cited (For data purposes)

1. Misra, Rishabh. "News Category Dataset." arXiv preprint arXiv:2209.11429 (2022).
2. Misra, Rishabh and Jigyasa Grover. "Sculpting Data for ML: The first act of Machine Learning." ISBN 9798585463570 (2021).